In [49]:
%matplotlib inline

In [50]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parallel Processing 

**1**. (30 points) Accelerating network bound procedures.

Python code to show PNG images available on the URL `http://people.duke.edu/~ccc14/misc/` is provided.

- Write a funciton `download_one(url, path)` that downloads one image given a `url` and saves the image to `path` (5 points)
    
For the exercises below, save the image file to the local directory using the same fileanme as on `http://people.duke.edu/~ccc14/misc/`. You can just use the `%%time` magic to time the downloads.
 
- Write a for loop to download all images and time how long it takes (5 points)
- Write a function that uses `concurrent.futures` and a thread pool with 4 threads to download all images and time how long it takes (5 points)
- Write a function that uses `multiprocessing` and a process pool with 4 processes to download all images and time how long it takes (5 points)
- Write a function that uses `ipyparallel` and a process pool with 4 processes to download all images and time how long it takes (10 points)

In [51]:
import requests
from bs4 import BeautifulSoup

def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') 
            if node.get('href').endswith(ext)]

site = 'http://people.duke.edu/~ccc14/misc/'
ext = 'png'
for i, file in enumerate(listFD(site, ext)):
    print(file)

http://people.duke.edu/~ccc14/misc/250px-002Ivysaur.png
http://people.duke.edu/~ccc14/misc/250px-003Venusaur.png
http://people.duke.edu/~ccc14/misc/250px-004Charmander.png
http://people.duke.edu/~ccc14/misc/250px-005Charmeleon.png
http://people.duke.edu/~ccc14/misc/250px-006Charizard.png
http://people.duke.edu/~ccc14/misc/250px-008Wartortle.png
http://people.duke.edu/~ccc14/misc/250px-009Blastoise.png
http://people.duke.edu/~ccc14/misc/250px-012Butterfree.png
http://people.duke.edu/~ccc14/misc/250px-015Beedrill.png
http://people.duke.edu/~ccc14/misc/250px-016Pidgey.png
http://people.duke.edu/~ccc14/misc/250px-017Pidgeotto.png
http://people.duke.edu/~ccc14/misc/250px-018Pidgeot.png
http://people.duke.edu/~ccc14/misc/250px-019Rattata.png
http://people.duke.edu/~ccc14/misc/250px-020Raticate.png
http://people.duke.edu/~ccc14/misc/250px-021Spearow.png
http://people.duke.edu/~ccc14/misc/250px-022Fearow.png
http://people.duke.edu/~ccc14/misc/250px-023Ekans.png
http://people.duke.edu/~ccc14/mi

### Function to download a single image at URL to PATH

In [52]:
def download_one(url,path):
    import requests
    r = requests.get(url, stream=True)
    with open(path, 'wb') as f:
        f.write(r.raw.read())

### Using for loop

In [53]:
files = listFD(site, ext)

In [54]:
%%time

for url in files:
    file_name = url.split('/')[-1]
    download_one(url, file_name)

CPU times: user 348 ms, sys: 96 ms, total: 444 ms
Wall time: 8.59 s


### Using a thread pool

In [55]:
%%time
from concurrent.futures import ThreadPoolExecutor

file_path = [(url,url.split('/')[-1]) for url in files]
with ThreadPoolExecutor(max_workers=4) as pool:
    pool.map(lambda x: download_one(x[0],x[1]), file_path)

CPU times: user 332 ms, sys: 116 ms, total: 448 ms
Wall time: 2.75 s


### Using a process pool

In [56]:
%%time

from multiprocessing import Pool

file_path = [(url,url.split('/')[-1]) for url in files]
with Pool(processes=4) as pool:
    pool.starmap(download_one, file_path)

CPU times: user 8 ms, sys: 32 ms, total: 40 ms
Wall time: 2.34 s


### Using `ipyparallel`

In [57]:
%%time

from ipyparallel import Client

rc = Client()
dv = rc[:]
path = [url.split('/')[-1] for url in files]
res = dv.map_sync(download_one, files, path)

CPU times: user 56 ms, sys: 32 ms, total: 88 ms
Wall time: 2.18 s
